<a href="https://colab.research.google.com/github/davidlealo/TalkTrack/blob/main/TalkTrack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TalkTrack

Carga de las librerías a utilizar

In [1]:
!pip install opencv-python-headless numpy openai-whisper sounddevice moviepy torch torchvision ultralytics pyaudio


  Using cached PyAudio-0.2.14.tar.gz (47 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for pyaudio (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pyaudio
Failed to build pyaudio
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (pyaudio)


### Programa de captura de imagen y audio

In [3]:
from ultralytics import YOLO
import cv2
import numpy as np
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from IPython.display import display, Javascript, clear_output
from base64 import b64decode
import time
from datetime import datetime
import whisper
import sounddevice as sd
import scipy.io.wavfile as wav
import threading
from queue import Queue

# Load YOLO model (YOLOv8n for lightweight detection)
model = YOLO("yolov8n.pt")
whisper_model = whisper.load_model("base")

# Audio capture and processing setup
CHUNK_DURATION = 5  # seconds
SAMPLE_RATE = 16000
CHANNELS = 1
AUDIO_QUEUE = Queue()
TRANSCRIPTION = []

def audio_capture():
    print("Recording audio...")
    def callback(indata, frames, time, status):
        if status:
            print(f"Audio status: {status}")
        AUDIO_QUEUE.put(indata.copy())

    with sd.InputStream(samplerate=SAMPLE_RATE, channels=CHANNELS, callback=callback):
        while not shutdown_signal:
            sd.sleep(1000)
    print("Audio recording stopped.")

def transcribe_audio():
    print("Starting transcription...")
    while not shutdown_signal or not AUDIO_QUEUE.empty():
        frames = []
        try:
            while len(frames) < SAMPLE_RATE * CHUNK_DURATION:
                chunk = AUDIO_QUEUE.get(timeout=1)
                frames.append(chunk)
        except:
            break

        audio_data = np.concatenate(frames, axis=0)
        wav.write("temp_audio.wav", SAMPLE_RATE, audio_data)

        result = whisper_model.transcribe("temp_audio.wav")
        TRANSCRIPTION.append(result["text"])
        print("Transcription:", result["text"])

    print("Transcription thread terminated.")

# Function to convert JavaScript object into an OpenCV image
def js_to_image(js_reply):
    try:
        image_bytes = b64decode(js_reply.split(',')[1])
        jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
        img = cv2.imdecode(jpg_as_np, flags=1)
        return img
    except Exception as e:
        print(f"Error decoding image: {e}")
        return None

# JavaScript for webcam streaming with shutdown control
def video_stream():
    display(Javascript('''
        var video;
        var div = null;
        var stream;
        var captureCanvas;
        var shutdown = false;

        function removeDom() {
           try {
               if (stream) {
                   stream.getTracks().forEach(track => track.stop());
               }
               if (video) {
                   video.remove();
               }
               if (div) {
                   div.remove();
               }
               video = null;
               div = null;
               stream = null;
               captureCanvas = null;
               console.log("DOM elements removed and camera stopped.");
           } catch (err) {
               console.error("Error removing DOM elements:", err);
           }
        }

        async function createDom() {
            if (div !== null) {
                return stream;
            }

            div = document.createElement('div');
            div.style.border = '2px solid black';
            div.style.padding = '3px';
            div.style.width = '100%';
            div.style.maxWidth = '640px';
            document.body.appendChild(div);

            video = document.createElement('video');
            video.style.display = 'block';
            video.width = div.clientWidth;
            video.setAttribute('playsinline', '');

            stream = await navigator.mediaDevices.getUserMedia({video: true});
            video.srcObject = stream;
            await video.play();

            div.appendChild(video);

            captureCanvas = document.createElement('canvas');
            captureCanvas.width = 640;
            captureCanvas.height = 480;

            const stopButton = document.createElement('button');
            stopButton.textContent = 'Stop Stream';
            stopButton.style.marginTop = '10px';
            stopButton.onclick = () => {
                shutdown = true;
                google.colab.kernel.invokeFunction('notebook.stopStream', [], {}).catch(err => {
                    console.error("Error invoking stopStream:", err);
                });
                removeDom();
            };
            div.appendChild(stopButton);

            return stream;
        }

        async function captureFrame() {
            if (!div) await createDom();
            if (shutdown) {
                removeDom();
                return 'shutdown';
            }
            captureCanvas.getContext('2d').drawImage(video, 0, 0, captureCanvas.width, captureCanvas.height);
            return captureCanvas.toDataURL('image/jpeg');
        }

        async function streamLoop() {
            try {
                while (!shutdown) {
                    const frame = await captureFrame();
                    await google.colab.kernel.invokeFunction('notebook.processFrame', [frame], {}).catch(err => {
                        console.error("Error invoking processFrame:", err);
                    });
                    await new Promise(resolve => setTimeout(resolve, 100));
                }
            } catch (err) {
                console.error("Error in streamLoop:", err);
                removeDom();
            } finally {
                removeDom();
                console.log("Stream loop exited.");
            }
        }

        streamLoop();
    '''))

# Function to process frames sent from JavaScript
from google.colab import output

current_frame = None
shutdown_signal = False

def process_frame(data):
    global current_frame
    try:
        current_frame = data[0]  # Adjust to read from array
    except Exception as e:
        print(f"Error processing frame: {e}")

def stop_stream():
    global shutdown_signal
    shutdown_signal = True
    print(f"Stop stream called at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

output.register_callback('notebook.processFrame', process_frame)
output.register_callback('notebook.stopStream', stop_stream)

# Function to capture webcam frames and run YOLO inference
def capture_and_detect():
    video_stream()
    global current_frame, shutdown_signal
    audio_thread = threading.Thread(target=audio_capture)
    transcription_thread = threading.Thread(target=transcribe_audio)

    audio_thread.start()
    transcription_thread.start()

    try:
        while not shutdown_signal:
            if current_frame is None:
                time.sleep(0.1)
                continue

            if current_frame == 'shutdown':
                print(f"Stream stopped by user at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
                break

            frame = js_to_image(current_frame)
            if frame is None:
                print("No valid frame received. Skipping.")
                continue

            results = model.predict(source=frame, conf=0.5)

            clear_output(wait=True)  # Clear the previous output
            for result in results:
                annotated_frame = result.plot()
                cv2_imshow(annotated_frame)

            current_frame = None  # Reset frame to wait for the next

        print("Detection stopped.")

    except KeyboardInterrupt:
        print("Keyboard interrupt detected. Stopping stream.")
        shutdown_signal = True

    except Exception as e:
        print("Error during detection:", e)

    finally:
        # Ensure all DOM elements are cleaned up when the function exits
        try:
            eval_js('removeDom();')
            print("DOM elements removed and resources cleaned up.")
        except Exception as cleanup_error:
            print("Error during final DOM cleanup:", cleanup_error)
        shutdown_signal = False  # Reset the shutdown signal
        audio_thread.join()
        transcription_thread.join()
        print("Shutdown signal reset. Program terminated.")
        with open("transcription.txt", "w") as f:
            f.write("\n".join(TRANSCRIPTION))
        print("Transcription saved to 'transcription.txt'.")

if __name__ == "__main__":
    print("Starting YOLO detection. Click on 'Stop Stream' to stop.")
    capture_and_detect()


OSError: PortAudio library not found